Simple example to see if the package is functioning properly.

In [1]:
import os
from escher.utils.dataset_loader import get_dataset as get_image_dataset
from escher.inference.model_ops import get_model, get_dataset
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

/var/local/atharvas/miniconda3/envs/escher-dev/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
dataset_name = "nabirds"
clip_model_name = "ViT-L/14"
device = "cuda:5"
start_iteration = 0
salt = ""
image_size = 224

In [7]:
kwargs = dict()
test_dataset = get_image_dataset(
    clip_model_name,
    dataset_name,
    image_size=image_size,
    transform=False,
    val_only=True,
)
train_clip_dataset, test_clip_dataset, val_clip_dataset = get_dataset(
    dataset_name,
    clip_model_name,
    device=device,
    use_open_clip=kwargs.get("use_open_clip", False),
)
classes = train_clip_dataset.classes

model, descriptors = get_model(
    algorithm="zero_shot",
    dataset_name=dataset_name,
    device=device,
    log_iteration=start_iteration,
    salt=salt,
    return_descriptors=True,
    clip_model_name=clip_model_name,
    dataset=val_clip_dataset,
    **kwargs,
)


/var/local/atharvas/f/escher/escher/cbd_utils/nabirds.py:48: RuntimeWarning: The use of the download flag is deprecated, since the dataset is no longer publicly accessible.
  warnings.warn(msg, RuntimeWarning)


Using conversational history
Loaded model from iteration 0 onto device cuda:5
using clip model: ViT-L/14 use_open_clip False


In [8]:
model.library.get_attributes()[:10], len(model.library.get_attributes()) / len(train_clip_dataset.classes)

(['white belly and underparts',
  'black bill with pink or orange "grinning patch"',
  'often perched on top of cacti or other plants',
  'fleshy, red wattle hanging from the throat',
  'white patch on the throat and sides of the neck',
  'a medium-sized bird of prey',
  'dark brown or black back and wings',
  'short, fan-shaped tail',
  'yellow-green rump and undertail coverts',
  'found in wooded areas or gardens with trees'],
 4.367567567567567)

In [9]:
scores = model.calculate_clip_scores(test_clip_dataset.images)
test_predictions = model.predict(test_clip_dataset.images, clip_scores=scores)
acc = ((test_predictions == test_clip_dataset.labels).sum() / len(test_clip_dataset.labels)).item() * 100

print("Accuracy of baseline", acc)

getting text embeddings: 100%|██████████| 3/3 [00:02<00:00,  1.46it/s]

Accuracy of baseline 53.57142686843872
